This notebook needs to be updated!!

In [3]:
import os
os.chdir("../..")
print(os.getcwd())

C:\Users\dietr004\Documents\PhD\computational mass spectrometry\WP1\MS2LDA


In [4]:
from MS2LDA.Preprocessing.load_and_clean import load_mgf
import matchms.filtering as msfilters

from MS2LDA.Add_On.Spec2Vec.annotation import calc_embeddings
from MS2LDA.Add_On.Spec2Vec.annotation import load_s2v_and_library

from tqdm import tqdm

from rdkit import Chem

import numpy as np
import sqlite3
import pickle

## Negative Mode

In [3]:
spectra_path = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\Spec2Struc\MS2LDA\Add_On\Spec2Vec\neg_model\cleaned_libraries.mgf"

In [4]:
#path_model = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\Spec2Struc\MS2LDA\Add_On\Spec2Vec\model\020724_Spec2Vec_CleanedLibraries.model"
path_model = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\Spec2Struc\MS2LDA\Add_On\Spec2Vec\neg_model\150724_Spec2Vec_neg_CleanedLibraries.model"
#path_model = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\Spec2Struc\MS2LDA\Add_On\Spec2Vec\spec2vec_AllPositive_ratio05_filtered_201101_iter_15.model"
path_library = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\Spec2Struc\MS2LDA\Add_On\Spec2Vec\library.pkl" # were are not using the library, so it can be any dummy pickle file

In [5]:
%%time 
s2v_similarity, _ = load_s2v_and_library(path_model, path_library)

CPU times: total: 13.2 s
Wall time: 13.3 s


In [6]:
%%time
spectra = load_mgf(spectra_path)

cleaned_spectra = []
smiles = []
embeddings = []


for spectrum in tqdm(spectra):
    # metadata filters
    spectrum = msfilters.default_filters(spectrum)
    spectrum = msfilters.add_retention_index(spectrum)
    spectrum = msfilters.add_retention_time(spectrum)
    spectrum = msfilters.require_precursor_mz(spectrum)

    # normalize and filter peaks
    spectrum = msfilters.normalize_intensities(spectrum)
    spectrum = msfilters.select_by_relative_intensity(spectrum, 0.001, 1)
    spectrum = msfilters.select_by_mz(spectrum, mz_from=0.0, mz_to=1000.0)
    spectrum = msfilters.reduce_to_number_of_peaks(spectrum, n_max=500)
    spectrum = msfilters.require_minimum_number_of_peaks(spectrum, n_required=3)
    spectrum = msfilters.add_losses(spectrum)

    if spectrum: 
        
        smi = spectrum.get("smiles")
        mol = Chem.MolFromSmiles(smi)
        Chem.RemoveStereochemistry(mol)
        smi2D = Chem.MolToSmiles(mol)
        
        spectrum.set("smiles", smi2D)
        cleaned_spectra.append(spectrum)
        
        
        smiles.append(smi2D)
        
        embedding = calc_embeddings(s2v_similarity, [spectrum])
        embeddings.append(embedding)

145594it [22:04, 109.88it/s]

CPU times: total: 22min 4s
Wall time: 22min 4s


In [7]:
print(f"Amount of spectra: {len(cleaned_spectra)}")

Amount of spectra: 132950


In [8]:
import pandas as pd
import pickle

In [9]:
new_library = pd.DataFrame({
    "embeddings": embeddings,
    "smiles": smiles,
    "spectra": cleaned_spectra,
})

In [10]:
with open(r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\Spec2Struc\MS2LDA\Add_On\Spec2Vec\neg_model\new_library.pkl", "wb") as outpkl:
    pickle.dump(new_library, outpkl)

---

This is for the "old" dataset where I only have the spectra stored together with the embeddings and smiles

In [11]:
import pickle

In [12]:
with open(r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\Spec2Struc\MS2LDA\Add_On\Spec2Vec\library.pkl", "rb") as inpkl:
    old_dataset = pickle.load(inpkl)

In [15]:
path_model = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\Spec2Struc\MS2LDA\Add_On\Spec2Vec\model\020724_Spec2Vec_CleanedLibraries.model"
path_library = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\Spec2Struc\MS2LDA\Add_On\Spec2Vec\library.pkl" # were are not using the library, so it can be any dummy pickle file

In [16]:
%%time 
s2v_similarity, _ = load_s2v_and_library(path_model, path_library)

CPU times: total: 21.7 s
Wall time: 27 s


In [17]:
%%time

spectra = []
smiles = []
embeddings = []


for spectrum in tqdm(old_dataset.spectra):
    spectra.append(spectrum)
    
    smi = spectrum.get("smiles")
    smiles.append(smi)
    
    embedding = calc_embeddings(s2v_similarity, [spectrum])
    embeddings.append(embedding)

100%|████████████████████████████████████████████████████████████████████████| 283261/283261 [04:32<00:00, 1040.05it/s]

CPU times: total: 4min 31s
Wall time: 4min 32s


In [19]:
print(f"Amount of spectra: {len(spectra)}")

Amount of spectra: 283261


In [20]:
import pandas as pd

In [21]:
old_library = pd.DataFrame({
    "embeddings": embeddings,
    "smiles": smiles,
    "spectra": spectra,
})

In [22]:
with open(r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\Spec2Struc\MS2LDA\Add_On\Spec2Vec\model\library.pkl", "wb") as outpkl:
    pickle.dump(old_library, outpkl)

## Positive Mode

In [18]:
#spectra_path_pos = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\WP1\MS2LDA\MS2LDA\Add_On\Spec2Vec\model_positive_mode_fast\positive_train_data\massspecgym.mgf"
spectra_path_pos = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\WP1\MS2LDA\MS2LDA\Add_On\Spec2Vec\model_positive_mode\positive_train_data\cleaned_libraries.mgf"
#path_model_pos = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\WP1\MS2LDA\MS2LDA\Add_On\Spec2Vec\model_positive_mode_fast\150225_Spec2Vec_pos_MassSpecGym.model"
path_model_pos = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\WP1\MS2LDA\MS2LDA\Add_On\Spec2Vec\model_positive_mode_fast\150225_Spec2Vec_pos_CleanedLibraries.model"
dummy_pkl_file = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\WP1\MS2LDA\MS2LDA\Add_On\Spec2Vec\model_positive_mode\positive_s2v_library.pkl" # were are not using the library, so it can be any dummy pickle file

In [19]:
%%time 
s2v_similarity, _ = load_s2v_and_library(path_model_pos, dummy_pkl_file)

CPU times: total: 25.6 s
Wall time: 28.7 s


In [20]:
%%time
spectra_pos = load_mgf(spectra_path_pos)

cleaned_spectra_pos = []
embeddings_pos = []


for spectrum in tqdm(spectra_pos):
    # metadata filters
    spectrum = msfilters.default_filters(spectrum)
    spectrum = msfilters.add_retention_index(spectrum)
    spectrum = msfilters.add_retention_time(spectrum)
    spectrum = msfilters.require_precursor_mz(spectrum)

    # normalize and filter peaks
    spectrum = msfilters.normalize_intensities(spectrum)
    spectrum = msfilters.select_by_relative_intensity(spectrum, 0.001, 1)
    spectrum = msfilters.select_by_mz(spectrum, mz_from=0.0, mz_to=1000.0)
    spectrum = msfilters.reduce_to_number_of_peaks(spectrum, n_max=500)
    spectrum = msfilters.require_minimum_number_of_peaks(spectrum, n_required=3)

    if spectrum: 
        
        smi = spectrum.get("smiles")
        mol = Chem.MolFromSmiles(smi)
        Chem.RemoveStereochemistry(mol)
        smi2D = Chem.MolToSmiles(mol)
        
        spectrum.set("smiles", smi2D)
        cleaned_spectra_pos.append(spectrum)
        
        embedding = calc_embeddings(s2v_similarity, [spectrum])
        embeddings_pos.append(embedding)

502685it [32:43, 256.02it/s]

CPU times: total: 32min 37s
Wall time: 32min 43s


In [21]:
x,y,z = np.array(embeddings_pos).shape
x,y,z

(426280, 1, 300)

In [22]:
np.save("MS2LDA/Add_On/Spec2Vec/model_positive_mode_fast/150225_CleanedLibraries_Spec2Vec_pos_embeddings.npy", np.array(embeddings_pos).reshape(x,z))

In [23]:
def create_spectra_db_from_list(spectra_list, db_path):
    """Create an SQLite database to store spectra from a list of spectra objects."""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS spectra (
            id TEXT PRIMARY KEY,
            smiles TEXT,
            spectrum BLOB
        )
    ''')
    conn.commit()

    for i, spectrum in enumerate(spectra_list):
        spectrum_id = i
        smiles = spectrum.get('smiles')
        spectrum_blob = pickle.dumps(spectrum)
        cursor.execute('''
            INSERT OR REPLACE INTO spectra (id, smiles, spectrum)
            VALUES (?, ?, ?)
        ''', (spectrum_id, smiles, spectrum_blob))

    conn.commit()
    conn.close()

In [24]:
create_spectra_db_from_list(cleaned_spectra_pos, "MS2LDA/Add_On/Spec2Vec/model_positive_mode_fast/150225_CombLibraries_spectra_b.db")